# Case Study: Real-Time Credit Card Fraud Detection

## Background: real-time feature engineering with DBSP

**Feature engineering** is the process of transforming raw data into a set of features that can be used to improve the predictive accuracy of an ML model.  Features are often expressed as SQL queries that filter, transform, aggregate, and join raw data.  These queries are evaluated by an RDBMS, e.g., Postgres, and the results are fed to the ML model during training and inference stages.

**Real-time feature engineering** arises in applications where data arrives continuously and requires immediate analysis, such as fraud detection and anomaly detection.  The main challenge in this process is extracting features from constantly changing data.  Although simple cases can be handled by streaming analytics platforms like Flink, they fall short when it comes to complex SQL queries that feature engineers commonly use (we will see examples of such queries in this case study!).  A common workaround is to precompute features through periodic batch jobs in an RDBMS such as BigQuery and inject the precomputed features into the real-time data stream.  This approach allows arbitrary feauture queries but sacrifices **feature freshness**, resulting in poor prediction accuracy in many real-time ML applications since precomputed features do not reflect the latest input data.

**DBSP aims to provide the benefits of both worlds** by evaluating complex feature queries directly on streaming data, eliminating the need for batch jobs and delivering perfect feature freshness.

## About this case study

Our goal in this case study is two-fold:

1. To illustrate how ML engineers can invoke DBSP from a Jupyter notebook to evaluate feature extraction queries on streaming data during model training, testing, and inference.
1. To empirically prove that DBSP enhances prediction accuracy in real-time ML.  Specifically, we demonstrate that **both complex queries and data freshness are critical for achieving high accuracy in real-time ML applications**.

This case study is based on the credit card fraud detection solution published by the Google Cloud blog:
https://cloud.google.com/blog/products/data-analytics/how-to-build-a-fraud-detection-solution

In [1]:
import os
import tarfile
import gdown
from os import path
import pandas as pd
from datetime import datetime
import geopy
import geopy.distance
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_curve, roc_curve
from collections import Counter
from sklearn.utils import shuffle
import warnings
import urllib.request
import json
import tensorflow as tf
import time
import threading
import requests

# API URL
api_url = "http://localhost:8080"

warnings.filterwarnings('ignore')

train_url = "https://raw.githubusercontent.com/feldera/ci_datasets/0eed3caac45415f6cd531dc5cb2e7f531d09a823/train_ci.csv"
test_url = "https://raw.githubusercontent.com/feldera/ci_datasets/0eed3caac45415f6cd531dc5cb2e7f531d09a823/test_ci.csv"
simulation_url = "https://raw.githubusercontent.com/feldera/ci_datasets/0eed3caac45415f6cd531dc5cb2e7f531d09a823/simulation_ci.csv"
demographics_url = "https://github.com/feldera/ci_datasets/raw/e06f92fbd74f11d96ceb8b19c7692d05c7aca6d0/demographics.csv"

train_path = path.abspath("fraud_data/train.csv")
test_path = path.abspath("fraud_data/train.csv")
simulation_path = path.abspath("fraud_data/simulation_short2.csv")
demographics_path = path.abspath("fraud_data/demographics.csv")

def train_outpath(query: str):
    return path.abspath(f"fraud_data/{query}-train_output.csv")
    
def test_outpath(query: str):
    return path.abspath(f"fraud_data/{query}-test_output.csv")

def simulation_outpath(query: str):
    return path.abspath(f"fraud_data/{query}-simulation_output.csv")

def create_program(program_name: str, query: str):
    program_sql = """
    CREATE TABLE demographics (
        cc_num FLOAT64 NOT NULL,
        first STRING,
        gender STRING,
        street STRING,
        city STRING,
        state STRING,
        zip INTEGER,
        lat FLOAT64 NOT NULL,
        long FLOAT64 NOT NULL,
        city_pop INTEGER,
        job STRING,
        dob STRING
        --dob DATE
    );

    CREATE TABLE transactions (
        trans_date_trans_time TIMESTAMP NOT NULL,
        cc_num FLOAT64 NOT NULL,
        merchant STRING,
        category STRING,
        amt FLOAT64,
        trans_num STRING,
        unix_time INTEGER NOT NULL,
        merch_lat FLOAT64 NOT NULL,
        merch_long FLOAT64 NOT NULL,
        is_fraud INTEGER
    );
    """ + query

    # Create program
    response = requests.put(f"{api_url}/v0/programs/{program_name}", json={
        "description": "",
        "code": program_sql
    })
    response.raise_for_status()
    program_version = response.json()["version"]

    # Compile program
    print(f"Compiling program '{program_name}' (version: {program_version})...")
    requests.post(f"{api_url}/v0/programs/{program_name}/compile", json={"version": program_version}).raise_for_status()
    while True:
        status = requests.get(f"{api_url}/v0/programs/{program_name}").json()["status"]
        print(f"Program status: {status}")
        if status == "Success":
            break
        elif status != "Pending" and status != "CompilingRust" and status != "CompilingSql":
            raise RuntimeError(f"Failed program compilation with status {status}")
        time.sleep(5)
    print("done")
    
def wait_for_pipeline(pipeline_name: str):
    # Wait till the pipeline is completed
    while not requests.get(f"{api_url}/v0/pipelines/{pipeline_name}/stats") \
            .json()["global_metrics"]["pipeline_complete"]:
        time.sleep(1)
    
    requests.post(f"{api_url}/v0/pipelines/{pipeline_name}/shutdown").raise_for_status()

def run_query(program_name, transaction_path: str, output_file: str):
    # Connectors
    connectors = []
    for (connector_name, stream, url, is_input) in [
        ("demographics", 'DEMOGRAPHICS', demographics_path, True),
        ("transactions", 'TRANSACTIONS', transaction_path, True),
        ("features", 'FEATURES', output_file, False),
    ]:
        requests.put(f"{api_url}/v0/connectors/{connector_name}", json={
            "description": "",
            "config": {
               "format": { "name": "csv", "config": {} },
               "transport": { "name": "file", "config": { "path": url, } }
            }
        })
        connectors.append({ "connector_name": connector_name, "is_input": is_input, "name": connector_name, "relation_name": stream })
    
    # Create pipeline
    pipeline_name = program_name + "-pipeline"
    requests.put(f"{api_url}/v0/pipelines/{pipeline_name}", json={
        "description": "",
        "config": {"workers": 6},
        "program_name": program_name,
        "connectors": connectors,
    }).raise_for_status()
    
    # Start pipeline in paused state, so we don't miss any outputs
    print("(Re)starting pipeline...")
    requests.post(f"{api_url}/v0/pipelines/{pipeline_name}/shutdown").raise_for_status()
    while requests.get(f"{api_url}/v0/pipelines/{pipeline_name}").json()["state"]["current_status"] != "Shutdown":
        time.sleep(1)
    requests.post(f"{api_url}/v0/pipelines/{pipeline_name}/pause").raise_for_status()
    while requests.get(f"{api_url}/v0/pipelines/{pipeline_name}").json()["state"]["current_status"] != "Paused":
        time.sleep(1)
        
    print("Pipeline (re)started")

    # # Start listening for the output stream
    # features = requests.post(f"{api_url}/v0/pipelines/{pipeline_name}/egress/features?format=csv", stream = True)   

    # Run the pipeline
    requests.post(f"{api_url}/v0/pipelines/{pipeline_name}/start").raise_for_status()
    while requests.get(f"{api_url}/v0/pipelines/{pipeline_name}").json()["state"]["current_status"] != "Running":
        time.sleep(1)
    
    # Wait for the pipeline to complete. The thread will shutdown the pipeline when it's done,
    # allowing the iteration below to terminate.
    thread = threading.Thread(target=wait_for_pipeline(pipeline_name))
    thread.start()
        
#     full_line = ""
#     print(f"writing features to {output_file}")
#     with open(output_file, "w") as feature_file:
#         for line in features.iter_lines():
#             if line:
#                 full_line += line.decode('utf-8')
#                 # print(f"line: {line}")
#                 try:
#                     data = json.loads(full_line).get("text_data")
#                     full_line = ""
#                     if data is not None:
#                         feature_file.write(data + '\n')
#                 except ValueError as e:
#                     print("partial json")
#         if full_line != "":
#             print(f"leftover: {full_line}")

def show_data(cm, print_res = 0):
    tp = cm[1,1]
    fn = cm[1,0]
    fp = cm[0,1]
    tn = cm[0,0]
    if print_res == 1:
        global pr, rec, f1
        pr = tp/(tp+fp)
        rec=  tp/(tp+fn)
        f1 = (2*(pr*rec)/(pr+rec))
        print('Precision =     {:.3f}'.format(pr))
        print('Recall (TPR) =  {:.3f}'.format(rec))
        #print('Fallout (FPR) = {:.3f}'.format(fp/(fp+tn)))
        print('F1 = {:.3f}'.format(f1))
    return tp/(tp+fp), tp/(tp+fn), fp/(fp+tn)

def train(query_name):
    max_depth = 12
    n_estimators = 100

    traindata     = pd.read_csv(train_outpath(query_name), float_precision='round_trip')  
    train_dataset = shuffle(traindata)

    test_dataset     = pd.read_csv(test_outpath(query_name), float_precision='round_trip')  

    nb_cols = len(train_dataset.columns.tolist())

    X_train = train_dataset.iloc[:, 0:nb_cols - 2].values
    y_train = train_dataset.iloc[:, nb_cols-2].values.astype(int)        

    X_test = test_dataset.iloc[:, 0:nb_cols - 2].values
    y_test = test_dataset.iloc[:, nb_cols-2].values.astype(int)    


    model = XGBClassifier(max_depth = max_depth,  n_estimators = n_estimators, objective = 'binary:logistic')#, scale_pos_weight= estimate) 
    setattr(model, 'verbosity', 0)


    model.fit(X_train, y_train)

    # evaluate train data
    y_pred = model.predict(X_train)
    predictions = [round(value) for value in y_pred]
    cm = confusion_matrix(y_train, predictions)
    show_data(cm, print_res = 1)

    # evaluate for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    cm = confusion_matrix(y_test, predictions)
    show_data(cm, print_res = 1)

    accuracy = accuracy_score(y_test, predictions)
    print(f"XGBoost Accuracy for {query_name}: %.2f%%" % (accuracy * 100.0))
    return model



2024-03-24 22:32:13.290372: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 22:32:13.290595: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 22:32:13.292447: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 22:32:13.319425: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 22:32:13.852236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

## Prepare a feature extraction query

In [2]:
queries = [
    ("raw",
        """CREATE VIEW features as
        SELECT
            DAYOFWEEK(trans_date_trans_time) AS d,
            -- TIMESTAMPDIFF(YEAR, trans_date_trans_time, CAST(dob as TIMESTAMP)) AS age,
            "ST_DISTANCE"("ST_POINT"(long,lat), "ST_POINT"(merch_long,merch_lat)) AS distance,
            -- TIMESTAMPDIFF(MINUTE, trans_date_trans_time, last_txn_date) AS trans_diff,
            category,
            amt,
            state,
            job,
            unix_time,
            city_pop,
            merchant,
            is_fraud
        FROM (
            SELECT t1.*, t2.*
                   -- , LAG(trans_date_trans_time, 1) OVER (PARTITION BY t1.cc_num  ORDER BY trans_date_trans_time ASC) AS last_txn_date
            FROM  transactions AS t1
            JOIN  demographics AS t2
            ON t1.cc_num = t2.cc_num);"""),
    ("rolling_aggregates", """CREATE VIEW features as
        SELECT
            DAYOFWEEK(trans_date_trans_time) AS d,
            -- TIMESTAMPDIFF(YEAR, trans_date_trans_time, CAST(dob as TIMESTAMP)) AS age,
            "ST_DISTANCE"("ST_POINT"(long,lat), "ST_POINT"(merch_long,merch_lat)) AS distance,
            -- TIMESTAMPDIFF(MINUTE, trans_date_trans_time, last_txn_date) AS trans_diff,
            AVG(amt) OVER(
                PARTITION BY   CAST(cc_num AS NUMERIC)
                ORDER BY unix_time
                -- 1 week is 604800  seconds
               RANGE BETWEEN 604800  PRECEDING AND 1 PRECEDING) AS avg_spend_pw,
            AVG(amt) OVER(
                PARTITION BY  CAST(cc_num AS NUMERIC)
                ORDER BY unix_time
                -- 1 month(30 days) is 2592000 seconds
                RANGE BETWEEN 2592000 PRECEDING AND 1 PRECEDING) AS avg_spend_pm,
            IFNULL(AVG(amt) OVER(
                PARTITION BY CAST(cc_num AS NUMERIC), EXTRACT(DAY FROM trans_date_trans_time)
                ORDER BY unix_time
                RANGE BETWEEN 7776000 PRECEDING AND 1 PRECEDING), 0) AS avg_spend_p3m_over_d,
            COUNT(*) OVER(
                PARTITION BY  CAST(cc_num AS NUMERIC)
                ORDER BY unix_time
                -- 1 day is 86400  seconds
                RANGE BETWEEN 86400  PRECEDING AND 1 PRECEDING ) AS trans_freq_24,
            category,
            amt,
            state,
            job,
            unix_time,
            city_pop,
            merchant,
            is_fraud
        FROM (
            SELECT t1.*, t2.*
                   -- , LAG(trans_date_trans_time, 1) OVER (PARTITION BY t1.cc_num  ORDER BY trans_date_trans_time ASC) AS last_txn_date
            FROM  transactions AS t1
            JOIN  demographics AS t2
            ON t1.cc_num = t2.cc_num);""")]

for (name, query) in queries:
    create_program(name, query)

Compiling program 'raw' (version: 1)...
Program status: Success
done
Compiling program 'rolling_aggregates' (version: 3)...
Program status: Pending
Program status: CompilingRust
Program status: CompilingRust
Program status: CompilingRust
Program status: Success
done


## Compute features on training and testing datasets

In [3]:
for (name, query) in queries:
    run_query(name, train_path, train_outpath(name))
    print(f"Training pipeline for '{name}' finished")
    run_query(name, test_path, test_outpath(name))
    print(f"Test pipeline for '{name}' finished")

(Re)starting pipeline...
Pipeline (re)started
Training pipeline for 'raw' finished
(Re)starting pipeline...
Pipeline (re)started
Test pipeline for 'raw' finished
(Re)starting pipeline...
Pipeline (re)started
Training pipeline for 'rolling_aggregates' finished
(Re)starting pipeline...
Pipeline (re)started
Test pipeline for 'rolling_aggregates' finished


## Train XGBoost model

In [4]:
models = {}

for (name, query) in queries:
    models[name] = train(name)
    

Precision =     1.000
Recall (TPR) =  0.986
F1 = 0.993
Precision =     1.000
Recall (TPR) =  0.986
F1 = 0.993
XGBoost Accuracy for raw: 99.98%
Precision =     1.000
Recall (TPR) =  1.000
F1 = 1.000
Precision =     0.998
Recall (TPR) =  0.989
F1 = 0.994
XGBoost Accuracy for rolling_aggregates: 99.99%


## Inference

In [5]:
log_dir = 'logs/xgboost_fraud_detection'
os.makedirs('logs/xgboost_fraud_detection', exist_ok=True)
print(f"Log directory: {log_dir}") 

Log directory: logs/xgboost_fraud_detection


In [6]:
summary_writer = tf.summary.create_file_writer(log_dir)

2024-03-24 22:36:42.257392: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: dbsp
2024-03-24 22:36:42.257408: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: dbsp
2024-03-24 22:36:42.257461: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2024-03-24 22:36:42.257485: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 525.147.5


In [7]:
for (name, query) in queries:
    run_query(name, simulation_path, simulation_outpath(name))
    print(f"Simulation pipeline for '{name}' finished")

(Re)starting pipeline...
Pipeline (re)started
Simulation pipeline for 'raw' finished
(Re)starting pipeline...
Pipeline (re)started
Simulation pipeline for 'rolling_aggregates' finished


In [8]:
for (name, query) in queries:
    try:
        chunksize = 1024*10
        print(f"reading data from {simulation_outpath(name)}")
        simulation = pd.read_csv(simulation_outpath(name), iterator=True, chunksize=chunksize)
        for simulation_batch in simulation:
            nb_cols = len(simulation_batch.columns.tolist())        
            X_simulation = simulation_batch.iloc[:, 0:nb_cols - 2].values
            y_simulation = simulation_batch.iloc[:, nb_cols-2].values.astype(int)     

            y_pred = models[name].predict(X_simulation)    
            predictions = [round(value) for value in y_pred]
            cm = confusion_matrix(y_simulation, predictions)
            #show_data(cm, print_res = 1)    
            accuracy = accuracy_score(y_simulation, predictions)
            print("XGBoost Accuracy: %.2f%%" % (accuracy * 100.0))

    except Exception as err:
        print(f"Error: cannot read from the specified source {err}")

reading data from /home/leonid/projects/feldera/demo/demo_notebooks/fraud_data/raw-simulation_output.csv
XGBoost Accuracy: 83.05%
XGBoost Accuracy: 83.80%
XGBoost Accuracy: 82.53%
XGBoost Accuracy: 84.12%
XGBoost Accuracy: 83.81%
XGBoost Accuracy: 83.11%
XGBoost Accuracy: 82.92%
XGBoost Accuracy: 84.45%
XGBoost Accuracy: 84.53%
XGBoost Accuracy: 84.35%
XGBoost Accuracy: 84.08%
XGBoost Accuracy: 83.42%
XGBoost Accuracy: 84.10%
XGBoost Accuracy: 83.89%
XGBoost Accuracy: 85.12%
XGBoost Accuracy: 84.61%
XGBoost Accuracy: 84.68%
XGBoost Accuracy: 85.33%
XGBoost Accuracy: 91.21%
XGBoost Accuracy: 94.75%
XGBoost Accuracy: 94.67%
XGBoost Accuracy: 94.27%
XGBoost Accuracy: 94.79%
XGBoost Accuracy: 93.17%
XGBoost Accuracy: 92.45%
XGBoost Accuracy: 92.30%
XGBoost Accuracy: 92.83%
XGBoost Accuracy: 99.08%
XGBoost Accuracy: 99.63%
XGBoost Accuracy: 99.31%
XGBoost Accuracy: 99.26%
XGBoost Accuracy: 99.43%
XGBoost Accuracy: 99.35%
XGBoost Accuracy: 99.46%
XGBoost Accuracy: 99.62%
XGBoost Accuracy: 99

In [9]:
step = 0
with summary_writer.as_default():
    tf.summary.scalar('accuracy', accuracy, step= step)
    tf.summary.scalar('precision', pr, step= step)
    tf.summary.scalar('recall', rec, step= step)
    tf.summary.scalar('f1_score', f1, step= step)

In [10]:
# train_summary_writer = tf.summary.create_file_writer(os.path.join(log_dir, 'train'))
# test_summary_writer = tf.summary.create_file_writer(os.path.join(log_dir, 'test'))
# simulation_summary_writer = tf.summary.create_file_writer(os.path.join(log_dir, 'simulation'))

In [11]:
%load_ext tensorboard
%tensorboard --logdir logs/xgboost_fraud_detection

In [12]:
import matplotlib.pyplot as plt
#Printing AUC
from sklearn.metrics import roc_curve, auc
y_PredTest= model.predict_proba(X_simulation)

fp, tp,_ = roc_curve(y_simulation, y_pred)


plt.plot(fp, tp, label=" AUC TEST ="+str(auc(fp, tp)))
plt.plot([0,1],[0,1],'g--')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("AUC(ROC curve)")
plt.show()

NameError: name 'model' is not defined

In [ ]:

importance = model.feature_importances_

for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
plt.bar([x for x in range(len(importance))], importance)
plt.show()